In [21]:
import cv2
import os

# 동영상 파일 경로
folder_path = "C:/Users/User/Images"

# 저장할 이미지 폴더 경로
output_folder = "C:/Users/User/frame_split/pullup/pullco"

# 동영상 파일 형식
video_formats = ('.mp4', '.avi', '.mov')  # 필요에 따라 확장자 추가

# 폴더 내의 모든 파일 가져오기
file_list = os.listdir(folder_path)

# 동영상 파일 필터링 및 동영상 경로 리스트 만들기
video_paths = [os.path.join(folder_path, file) for file in file_list if file.lower().endswith(video_formats)]

# 특정 단어가 포함된 동영상 파일만 선택하기
video_paths_with_keyword = [path for path in video_paths if "pull" in path.lower()]

# 이미지 저장을 위한 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# 각 동영상 파일별로 처리
for video_path in video_paths_with_keyword:
    # 동영상 파일 열기
    cap = cv2.VideoCapture(video_path)

    # 동영상 파일이 정상적으로 열렸는지 확인
    if not cap.isOpened():
        print("Error: Could not open video.")
        continue

    # 동영상의 초당 프레임 수 가져오기
    fps = cap.get(cv2.CAP_PROP_FPS)

    # 0.5초마다 이미지 저장할 시간 간격
    interval = int(fps * 0.5)

    # 프레임 인덱스 초기화
    frame_index = 0

    # 이미지 저장 시간 초기화
    save_time = 0

    # 프레임 단위로 동영상을 읽어와 이미지로 저장
    while True:
        # 동영상 파일로부터 프레임 읽기
        ret, frame = cap.read()

        # 프레임을 모두 읽었거나 동영상 파일이 종료되면 반복문 탈출
        if not ret:
            break

        # 0.5초마다 이미지 저장
        if frame_index >= save_time:
            # 프레임 이미지 저장
            image_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(video_path))[0]}_frame_{frame_index}.jpg")
            cv2.imwrite(image_path, frame)

            # 이미지 저장 시간 업데이트
            save_time += interval

        # 다음 프레임 인덱스로 이동
        frame_index += 1

    # 동영상 파일 닫기
    cap.release()

print("Images saved successfully in the 'frame_split/pullup/pullco' folder.")


Images saved successfully in the 'frame_split/pullup/pullco' folder.


In [22]:
import os
import cv2
import math

#스쿼트
# MPII에서 각 파트 번호, 선으로 연결될 POSE_PAIRS
BODY_PARTS = { "Head": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "Chest": 14,
                "Background": 15 }

POSE_PAIRS = [ ["Head", "Neck"], ["Neck", "RShoulder"], ["RShoulder", "RElbow"],
                ["RElbow", "RWrist"], ["Neck", "LShoulder"], ["LShoulder", "LElbow"],
                ["LElbow", "LWrist"], ["Neck", "Chest"], ["Chest", "RHip"], ["RHip", "RKnee"],
                ["RKnee", "RAnkle"], ["Chest", "LHip"], ["LHip", "LKnee"], ["LKnee", "LAnkle"] ]

# 각 파일 path
protoFile = "C:/Users/User/pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "C:/Users/User/pose_iter_160000.caffemodel"

# 위의 path에 있는 network 불러오기
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

In [23]:
def calculate_angle(a, b, c):
    if None in (a, b, c):
        return None
    try:
        rad1 = math.atan2(a[1] - b[1], a[0] - b[0])
        rad2 = math.atan2(c[1] - b[1], c[0] - b[0])
        angle = abs(rad1 - rad2) * 180 / math.pi
        return angle
    except ZeroDivisionError:
        return None

def calculate_distance(point_a, point_b):
    if None in (point_a, point_b):
        return None
    x1, y1 = point_a
    x2, y2 = point_b
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

In [25]:
import os
import cv2
import math
import csv
import numpy as np

# 이미지 파일이 있는 폴더 경로
folder_path = "C:/Users/User/frame_split/pullup/pullco"

# 폴더 내의 모든 파일 가져오기
file_list = os.listdir(folder_path)

output_folder = "C:/Users/User/frame_point/pullup/pullco"

# CSV 파일 경로 설정
output_csv_path = os.path.join(output_folder, "all_points.csv")

# 이미지 파일 필터링 및 이미지 경로 리스트 만들기
image_paths = [os.path.join(folder_path, file) for file in file_list if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

# CSV 파일 열기 (append 모드로)
with open(output_csv_path, 'a', newline='') as csvfile:
    # CSV writer 생성
    csvwriter = csv.writer(csvfile)
    
    # 헤더 작성
    csvwriter.writerow(["Frame Name", "Point 0", "Point 1", "Point 2", "Point 3", "Point 4", "Point 5", "Point 6", "Point 7", "Point 8", "Point 9", "Point 10", "Point 11", "Point 12", "Point 13", "Point 14", "arm_angle_right","arm_angle_left", "shoulder_length","wrist_length", "Posture"])
    # 이미지 경로마다 반복하여 처리
    for image_path in image_paths:
        # 프레임 이름 추출
        frame_name = os.path.basename(image_path)
        
        # 이미지 읽어오기
        image = cv2.imread(image_path)
        # frame.shape = 불러온 이미지에서 height, width, color 받아옴
        imageHeight, imageWidth, _ = image.shape
        # network에 넣기위해 전처리
        inpBlob = cv2.dnn.blobFromImage(image, 1.0 / 255, (imageWidth, imageHeight), (0, 0, 0), swapRB=False, crop=False)
        image = cv2.resize(image, (imageWidth, imageHeight))
        # network에 넣어주기
        net.setInput(inpBlob)

        # 결과 받아오기
        output = net.forward()

        # 키포인트 검출시 이미지에 그려줌
        points = []
        for i in range(0, 15):
            # 해당 신체부위 신뢰도 얻음.
            probMap = output[0, i, :, :]

            # global 최대값 찾기
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # 원래 이미지에 맞게 점 위치 변경
            x = (imageWidth * point[0]) / output.shape[3]
            y = (imageHeight * point[1]) / output.shape[2]

            # 키포인트 검출한 결과가 0.3보다 크면(검출한곳이 위 BODY_PARTS랑 맞는 부위면) points에 추가, 검출했는데 부위가 없으면 None으로
            if prob > 0.3 :    
                points.append((int(x), int(y)))
            else :
                points.append(None)
        
                           
        chest_point = points[BODY_PARTS["Chest"]]
        shoulder_left = points[BODY_PARTS["LShoulder"]]
        shoulder_right = points[BODY_PARTS["RShoulder"]]
        elbow_left = points[BODY_PARTS["LElbow"]]
        elbow_right = points[BODY_PARTS["RElbow"]]
        wrist_left = points[BODY_PARTS["LWrist"]]
        wrist_right = points[BODY_PARTS["RWrist"]]
        knee_left = points[BODY_PARTS["LKnee"]]

       # 첫 번째 저장: 프레임 이름과 포인트들, 빈 칼럼들

        if chest_point and shoulder_left and shoulder_right and elbow_right and elbow_left and wrist_left and wrist_right and knee_left:
            # 무릎 각도 계산 (다리 사이의 각도)
            arm_angle_right = calculate_angle(chest_point, shoulder_right, elbow_right)
            arm_angle_left = calculate_angle(chest_point, shoulder_left, elbow_left)
            shoulder_length = calculate_distance(shoulder_right, shoulder_left)
            wrist_length = calculate_distance(wrist_right, wrist_left)
                  
            # 초록 선 그리기
            for pair in POSE_PAIRS:
                partA = pair[0]
                partB = pair[1]
                if points[BODY_PARTS[partA]] and points[BODY_PARTS[partB]]:
                    cv2.line(image, points[BODY_PARTS[partA]], points[BODY_PARTS[partB]], (0, 255, 0), 2)
        
            # 이미지 저장
            output_image_path = os.path.join(output_folder, "Images", frame_name)
            cv2.imwrite(output_image_path, image)

            # 각 프레임의 데이터를 CSV 파일에 추가
            csvwriter.writerow([frame_name] + points+[arm_angle_right,arm_angle_left, shoulder_length,wrist_length])

print("All points and images saved successfully.")


All points and images saved successfully.


In [29]:
output_image_path

'C:/Users/User/frame_point/pullup/pullco\\Images\\pullup1_frame_90.jpg'

In [26]:
import pandas as pd
import ast

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(output_csv_path)


# 데이터프레임의 한 열에서 y값을 출력하는 함수 정의
def get_y_value(coord_str):
    # 문자열을 파싱하여 튜플로 변환
    coord_tuple = ast.literal_eval(coord_str)
    # 튜플의 두 번째 요소인 y값을 반환
    return coord_tuple[1]

def find_min_y_frame(df, column_name):
    y_min = []
    min_y_frames = []
# 데이터프레임의 한 열에서 y값을 추출하여 출력
    for index, row in df.iterrows():
        y_value = get_y_value(row['Point 12'])  # 'Your_Column_Name'은 실제 열의 이름으로 바꿔주세요
        y_min.append(y_value)
        min_y_value = min(y_min)
        
    for index, y_value in enumerate(y_min):
        if y_value == min_y_value:
            min_y_frames.append(df.iloc[index]['Frame Name'])
    return min_y_frames



In [27]:
import ast
import shutil

result_folder = "frame_point/pullup/pullco/Images_co"
min_y_result=find_min_y_frame(df,'Point 12')


for frame_name in min_y_result:
        # 주어진 프레임에 해당하는 데이터프레임 추출
        frame_df = df[df['Frame Name'] == frame_name]
        
        # 프레임의 관절 포인트 추출
        points = frame_df.iloc[0][1:16]  # 프레임의 관절 포인트는 데이터프레임의 두 번째부터 열에 저장되어 있다고 가정
        
        # 관절 포인트 문자열을 튜플로 변환
        points = [ast.literal_eval(p) if pd.notna(p) and p != "None" else None for p in points]
        
        chest_point = points[BODY_PARTS["Chest"]]
        shoulder_left = points[BODY_PARTS["LShoulder"]]
        shoulder_right = points[BODY_PARTS["RShoulder"]]
        elbow_left = points[BODY_PARTS["LElbow"]]
        elbow_right = points[BODY_PARTS["RElbow"]]
        wrist_left = points[BODY_PARTS["LWrist"]]
        wrist_right = points[BODY_PARTS["RWrist"]]
        
        
        posture = ""
        if chest_point and shoulder_left and shoulder_right and elbow_right and elbow_left and wrist_left and wrist_right:
            arm_angle_right = calculate_angle(chest_point, shoulder_right, elbow_right)
            arm_angle_left = calculate_angle(chest_point, shoulder_left, elbow_left)
            shoulder_length = calculate_distance(shoulder_right, shoulder_left)
            wrist_length = calculate_distance(wrist_right, wrist_left)

            if ((shoulder_length * 2.0) >= wrist_length >= (shoulder_length * 1.5)):
                if ((51 >= arm_angle_right >= 41) & (51 >= arm_angle_left >= 41)):
                    posture = "정확한 자세"
                else:
                    posture = "올바르지 못한 자세"
            else:
                posture = "올바르지 못한 자세"
        
             # 이미지 불러오기 및 초록 선 그리기
            image_path = os.path.join(folder_path, frame_name)
            image = cv2.imread(image_path)

            for pair in POSE_PAIRS:
                partA = pair[0]
                partB = pair[1]
                if points[BODY_PARTS[partA]] and points[BODY_PARTS[partB]]:
                    cv2.line(image, points[BODY_PARTS[partA]], points[BODY_PARTS[partB]], (0, 255, 0), 2)


                for pair in POSE_PAIRS:
                    partA = pair[0]
                    partB = pair[1]
                    if points[BODY_PARTS[partA]] and points[BODY_PARTS[partB]]:
                        cv2.line(image, points[BODY_PARTS[partA]], points[BODY_PARTS[partB]], (0, 255, 0), 2)


            # 결과를 데이터프레임에 추가
            df.loc[df['Frame Name'] == frame_name, 'Posture'] = posture
            
             # 이미지 복사
            #src_image_path = os.path.join(folder_path, frame_name)
           #dst_image_path = os.path.join(result_folder, frame_name)
            #shutil.copyfile(src_image_path, dst_image_path)
            
# 업데이트된 데이터프레임 저장
df.to_csv(output_csv_path, index=False, encoding='cp949')

print("All points and images saved successfully.")

All points and images saved successfully.


In [ ]:
# 피드백
# wrist_length, shoulder_length 비교 : 손을 좀 넓게 잡아라
#arm_angle_left, right : 어느쪽 팔을 더 접혀야한다.